<a href="https://colab.research.google.com/github/twelvesense/first-repository/blob/master/ExplNode05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트: 가위바위보 분류기를 만들자

In [ ]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

2.8.0
1.21.6


1) 데이터 준비 

2) 딥러닝 네트워크 설계 

3) 학습

4) 테스트(평가)

5) 개선하기